In [ ]:
pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [ ]:

pip install -U langchain langchain-openai

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = 'api_key' #kicseréltem, hogy a githubon ne legyen benne a kulcs

In [ ]:
os.environ['OPENAI_API_KEY'] = 'api_key'

In [ ]:
pip install PyGithub

In [ ]:
pip install faiss-cpu


In [ ]:
pip install faiss-cpu

## Github repo feldolgozása és indexelés

In [8]:
import requests
from github import Github
from bs4 import BeautifulSoup
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
import faiss
import numpy as np

In [9]:
#GitHub Repoból kiszedi a szükséges dolgokat (kód, dokumentáció)
def clone_repo(github_token, repo_name):
    g = Github(github_token)
    repo = g.get_repo(repo_name)
    
    contents = repo.get_contents("")
    
    files = []
    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            contents.extend(repo.get_contents(file_content.path))
        else:
            if file_content.name.endswith(('.md', '.py', '.txt', '.json', '.yml', '.c', '.h', '.cpp', '.hpp')): # Ezeket a fájlokat fogadja el egyenlőre
                file_data = {
                    "file_name": file_content.name,
                    "file_content": requests.get(file_content.download_url).text
                }
                files.append(file_data)
    
    return files

In [10]:
#A Issue-k kiszedése a repo-ból
def fetch_issues(github_token, repo_name):
    g = Github(github_token)
    repo = g.get_repo(repo_name)
    
    issues = []
    for issue in repo.get_issues(state='open'):
        issue_data = {
            "file_name": f"issue_{issue.number}",
            "file_content": f"Title: {issue.title}\nDescription: {issue.body}"
        }
        issues.append(issue_data)
    
    return issues

In [11]:
#A fájlok kisebb darabokra bontása
def preprocess_files(files):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    documents = []
    
    for file in files:
        content = file['file_content']
        
        #Markdown fájlok esetén HTML kód eltávolítása
        if file['file_name'].endswith(('.md', '.html')):
            soup = BeautifulSoup(content, 'html.parser')
            content = soup.get_text()
        
        chunks = text_splitter.split_text(content)
        
        for chunk in chunks:
            documents.append(chunk)
    
    return documents

In [12]:
#Embeddingek generálása  OpenAI embeddings-el és FAISS indexeléssel
def create_embeddings(documents):
    embeddings = OpenAIEmbeddings()
    
    doc_embeddings = embeddings.embed_documents(documents)
    doc_embeddings_np = np.array(doc_embeddings)
    
    index = faiss.IndexFlatL2(doc_embeddings_np.shape[1])
    index.add(doc_embeddings_np)
    
    return index, doc_embeddings_np, documents

In [ ]:
#Egy konkrét repora alkalmazva a fenti funkciókat

github_token = "api_key"
repo_name = "pydantic/pydantic"
    
files = clone_repo(github_token, repo_name)
    
issues = fetch_issues(github_token, repo_name)
    
all_documents = files + issues

In [ ]:
documents = preprocess_files(all_documents)
    
index, doc_embeddings_np, documents = create_embeddings(documents)

## Retrieval

In [15]:
#A kérdés (Query) feldolgozása és a legrelevánsabb dokumentumok visszaadása
def retrieve_relevant_document(query, index, documents, embeddings, k=5):
    query_embedding = embeddings.embed_query(query)
    query_embedding_np = np.array(query_embedding).reshape(1, -1)
    D, I = index.search(query_embedding_np, k)
    
    relevant_docs = [documents[i] for i in I[0]]
    return relevant_docs

In [16]:
#Példa kérdés
query = "Does this program support Python 3.14?"
relevant_docs = retrieve_relevant_document(query, index, documents, OpenAIEmbeddings())
    
print(f"5 válasz legrevelánsabb dokumentum:")
for doc in relevant_docs:
        print(f"- {doc}")

5 válasz legrevelánsabb dokumentum:
- Title: Add support for Python 3.14
Description: First 3.14 beta release is [planned on 2025-05-06](https://peps.python.org/pep-0745/#release-schedule) and PEP 649/749 is almost fully implemented.

Considering the significant changes it provides to the runtime evaluation of type hints, we should add support to 3.14 and report any bugs/issues.
- ### Python, Pydantic & OS Version

```Text
sqlalchemy 1.4
pydantic 2.7.1
python 3.11.5
```
- ```Python

```

### Python, Pydantic & OS Version

```Text
2.10
```
- ```Text
Python 3.10.9
pydantic 2.0a4
Windows 10
```
- As it is being proposed as an official standard for Python, other editors can also easily add support for it.

And authors of other libraries similar to Pydantic can also easily adopt the standard right away (using the "Alternate Form") and get the benefits of these additional editor features.


## Generation

In [17]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [18]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [19]:
chain = prompt | llm

In [20]:
chain.invoke({"context":relevant_docs,"question":"Does this program support Python 3.14?"})

AIMessage(content='Yes, the program supports Python 3.14 as it mentions adding support for Python 3.14 and reporting any bugs/issues related to it.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 287, 'total_tokens': 318, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BI0hTQc1k9Jy3InGqaLvXchGYEUnI', 'finish_reason': 'stop', 'logprobs': None}, id='run-e0b0224f-cc5a-4f5c-bd2a-23c122fe794c-0', usage_metadata={'input_tokens': 287, 'output_tokens': 31, 'total_tokens': 318, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})